### Day 20 - Part 1:

What really matters here? 
- We don't care about the center, just the border, so that could be stored off? 
- From there, can iterate through and match stuff together

In [16]:
from collections import defaultdict
import numpy as np 

# Read in test data
filepath = "day20_test_data.txt"
with open(filepath) as fh:
    lines = [line.strip() for line in fh.readlines()]

# default dict that can be appended to
temp_dict = defaultdict(list)

In [17]:
# Prepare each image
for line in lines:
    if 'Tile' in line:
        img_num = line.split(' ')[1][:-1] # extract image number
    elif line == '':
        pass # bad input, skip
        
    else:
        temp_dict[img_num].append([sym for sym in line])

# Convert each image to a 2-D array & store in final dict
image_dict = {}

for num, img in temp_dict.items():
    
    # flatten & rebuild as array 
    flat_list = [item for sublist in img for item in sublist]
    
    # rebuild as an array of size row, col
    row = len(img) # total rows
    col = len(img[0]) # elements in a single row
    array = np.array(flat_list)
    array.shape = ((row,col))
    
    # store the important vals: left col, top row, right col, bottom row
    left_col = array[:,0]
    right_col = array[:,col-1]
    top_row = array[0,]
    bot_row = array[row-1,]
    
    # append all into dict
    image_dict[num] = [left_col, right_col, top_row, bot_row]

In [18]:
image_dict['2311']

[array(['.', '#', '#', '#', '#', '#', '.', '.', '#', '.'], dtype='<U1'),
 array(['.', '.', '.', '#', '.', '#', '#', '.', '.', '#'], dtype='<U1'),
 array(['.', '.', '#', '#', '.', '#', '.', '.', '#', '.'], dtype='<U1'),
 array(['.', '.', '#', '#', '#', '.', '.', '#', '#', '#'], dtype='<U1')]

#### Working test case:

- In the test case we know that 1951 connects to 2311 
- Specifically, 1951 right col matches to 2311 left col
- Each boundary (top, left, right, bot) can be rotated (compare to everything) or flipped (-1 oflist?)
    - should only need to flip once....doing twice would jsut replicate a process I think 
    
- **We would then just look for those that are missing two matches....these represent 4 corners**
  - I am happy to have considered this!

In [32]:
boundary = {}
idx = 0 # [left_col, right_col, top_row, bot_row], flip is add 4
for v in image_dict['1951']:
    print(v)
    print(np.flip(v))
    print("Next")
    
    # iterate through other images - this would count as a rotation
    for k,val in image_dict.items():
        if k == '1951':
            continue
        
        for border in val:
            if np.array_equal(border, v):
                print(f"{k} matched a border...")
                boundary[idx] = k
                idx += 1
            elif np.array_equal(border, np.flip(v)):
                print(f"{k} matched a border...")
                boundary[idx+4] = k
                idx += 1                

['#' '#' '.' '#' '.' '.' '#' '.' '.' '#']
['#' '.' '.' '#' '.' '.' '#' '.' '#' '#']
Next
['.' '#' '#' '#' '#' '#' '.' '.' '#' '.']
['.' '#' '.' '.' '#' '#' '#' '#' '#' '.']
Next
2311 matched a border...
['#' '.' '#' '#' '.' '.' '.' '#' '#' '.']
['.' '#' '#' '.' '.' '.' '#' '#' '.' '#']
Next
2729 matched a border...
['#' '.' '.' '.' '#' '#' '.' '#' '.' '.']
['.' '.' '#' '.' '#' '#' '.' '.' '.' '#']
Next


### Bringing It All Together:

- Process above will build out matches
- Next step is to iterate through all images & build out a dictionary with their matches
- Then identify the 4 corners from those that only have 2 matches 

In [41]:
boundary_dict = {}

for num, bords in image_dict.items():
    
    boundary_dict[num] = [] # list to store our info
    for v in bords:
        # iterate through other images - this would count as a rotation
        for k,val in image_dict.items():
            if k == num:
                continue

            for border in val:
                if np.array_equal(border, v):
                    #print(f"{k} matched a border...")
                    boundary_dict[num].append(k)
                    idx += 1
                elif np.array_equal(border, np.flip(v)):
                    #print(f"{k} matched a border...")
                    boundary_dict[num].append(k)
                    idx += 1
                else:
                    continue

In [42]:
boundary_dict

{'2311': ['1951', '3079', '1427'],
 '1951': ['2311', '2729'],
 '1171': ['1489', '2473'],
 '1427': ['2729', '2473', '1489', '2311'],
 '1489': ['2971', '1171', '1427'],
 '2473': ['1171', '3079', '1427'],
 '2971': ['1489', '2729'],
 '2729': ['1427', '2971', '1951'],
 '3079': ['2311', '2473']}

In [44]:
prod = 1
for k,v in boundary_dict.items():
    if len(v) == 2:
        print(k)
        prod *= int(k)
        
print(f"Product of four corners = {prod}")

1951
1171
2971
3079
Product of four corners = 20899048083289
